<a href="https://colab.research.google.com/github/msjun23/Deep-Learning-from-Scratch/blob/main/Chapter5/2_activation_function_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

신경망을 구성하는 계층 각각을 하나의 클래스로 구현한다.

# ReLU layer
활성화 함수로 사용된는 **ReLU** 계층의 수식은 다음과 같다.
$$y=\left\{\begin{matrix}
x\;\;(x>0)\\ 
0\;\;(x\leq 0)
\end{matrix}\right.$$

위의 식에서 $x$에 대한 $y$의 미분은 아래와 같다.
$$\frac{\partial y}{\partial x}=\left\{\begin{matrix}
1\;\;(x>0)\\ 
0\;\;(x\leq 0)
\end{matrix}\right.$$

In [1]:
# ReLU 함수를 가진 계층 구현
class ReLU:
  def __init__(self):
    self.mask = None

  def forward(self, x):
    self.mask = (x <= 0)
    out = x.copy()
    out[self.mask] = 0

    return 0

  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout

    return dx

In [2]:
 import numpy as np

 x = np.array([[1.0, -0.5], [-2.0, 3.0]])
 print(x)

 mask = (x <= 0)
 print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


# Sigmoid layer

Sigmoid 함수는 다음과 같다.
$$y=\frac{1}{1+\text{exp}(-x)}$$

Simgoid 함수를 $x$에 대해 미분하면 아래와 같다.
$$\frac{\partial y}{\partial x}=\frac{e^{-x}}{(1+e^{-x})^2}
\\=\frac{1+e^{-x}-1}{(1+e^{-x})^2}
\\=y(1-y)$$

In [3]:
# Sigmoid 함수를 가진 계층 구현
class Sigmoid():
  def __init__(self):
    self.out = None
  
  def forward(self, x):
    out = 1 / (1 + np.exp(-x))
    self.out = out

    return out

  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out

    return dx

# Affine layer

신경망에서 순전파 때 수행하는 행렬의 곱은 **어파인 변환**(affine transformation)이라고 한다. 이러한 어파인 변환을 수행하는 처리를 Affine 계층으로 구현한다.

행렬 데이터에 대해 역전파를 전개하면 아래와 같다.
$$\frac{\partial L}{\partial X}=\frac{\partial L}{\partial Y}\cdot W^T
\\ \frac{\partial L}{\partial W}=X^T\cdot\frac{\partial L}{\partial Y}$$

$W^T$는 **전치행렬**을 의미한다. 전치행렬이란 행렬 $(i, j)$ 위치의 원소를 $(j, i)$ 위치로 이동하는 것을 말한다. 

In [4]:
# 데이터 N개를 묶어 순전파 진행을 하는 경우
# 배치용 Affine 계층
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

print(X_dot_W)

print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [5]:
# 역전파
dY = np.array([[1, 2, 3], [4, 5, 6]])
print(dY)

dB = np.sum(dY, axis=0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [6]:
# Affine 구현
class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None
    
  def forward(self, x):
    self.x = x
    out = np.dot(x, self.W) + self.b

    return out

  def backward(self, dout):
    dx = np.dot(dout, self.W.T)
    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis=0)

    return dx

# Softmax-with-Loss layer

소프트맥스 함수는 주로 출력층에서 사용한다. 소프트맥스는 입력값을 정규화하여 출력한다. 소프트맥스는 문제를 확률적으로 접근할 수 있게 해준다.

소프트맥스 함수와 교차 엔트로피 오차 함수를 같이 사용한다. 그 결과 소프트맥스 계층의 역전파는(y - t)로 깔끔하게 나온다. 이는 교차 엔트로피 오차 함수가 애초에 그렇게 설계되었기 때문이다.

In [7]:
# Softmax-with-loss 계층 구현
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None    # 손실
    self.y = None       # softmax의 출력
    self.t = None       # 정답 레이블(원-핫 벡터)

  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)

    return self.loss

  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size

In [8]:
# 복습 : softmax, cross_entropy_error 구현
def softmax(a):
  c = np.max(a)
  exp_a = np.exp(a - c)   # 오버플로 대책
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a

  return y

def cross_entropy_error(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  batch_size = y.shape[0]
  return -np.sum(t * np.log(y + 1e-7)) / batch_size